In [ ]:
## Objective:
# - merge unpivot data with calendar and calendar events.
# - clean up calendar cols for machine learning

In [5]:
import pandas as pd


train_clean = pd.read_csv('../data/interim/train_clean_1.csv')

In [6]:
df_cal = pd.read_csv('../data/raw/calendar.csv')
df_events = pd.read_csv('../data/raw/calendar_events.csv')

In [7]:
train_clean

,item_id,dept_id,cat_id,store_id,state_id,d,items_sold
0,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
1,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
2,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
3,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
4,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
...,...,...,...,...,...,...,...
46985085,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,d_1541,2
46985086,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,d_1541,0
46985087,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,d_1541,0
46985088,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,d_1541,0


In [8]:
### merge train with calendar

train_clean_2 = pd.merge(train_clean, df_cal, on='d', how='left')

In [12]:
df_events

,date,event_name,event_type
0,2011-02-06,SuperBowl,Sporting
1,2011-02-14,ValentinesDay,Cultural
2,2011-02-21,PresidentsDay,National
3,2011-03-09,LentStart,Religious
4,2011-03-16,LentWeek2,Religious
...,...,...,...
162,2016-05-30,MemorialDay,National
163,2016-06-02,NBAFinalsStart,Sporting
164,2016-06-07,Ramadan starts,Religious
165,2016-06-19,Father's day,Cultural


In [13]:
train_clean_2 = pd.merge(train_clean_2, df_events, on='date', how='left')

In [17]:
train_clean_2

,item_id,dept_id,cat_id,store_id,state_id,d,items_sold,date,wm_yr_wk,event_name,event_type
0,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,NaN,NaN
1,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,NaN,NaN
2,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,NaN,NaN
3,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,NaN,NaN
4,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
47107045,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,d_1541,2,2015-04-18,11512,NaN,NaN
47107046,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,d_1541,0,2015-04-18,11512,NaN,NaN
47107047,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,d_1541,0,2015-04-18,11512,NaN,NaN
47107048,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,d_1541,0,2015-04-18,11512,NaN,NaN


In [16]:
train_clean_2.to_parquet('../data/interim/train_clean_2.parquet', index=False)

In [18]:
df_sales = pd.read_csv('../data/raw/items_weekly_sell_prices.csv')
df_sales

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26
...,...,...,...,...
6841116,WI_3,FOODS_3_827,11617,1.00
6841117,WI_3,FOODS_3_827,11618,1.00
6841118,WI_3,FOODS_3_827,11619,1.00
6841119,WI_3,FOODS_3_827,11620,1.00


In [19]:
train_clean_3 = pd.merge(train_clean_2, df_sales, left_on = ['store_id', 'item_id', 'wm_yr_wk'], right_on = ['store_id', 'item_id', 'wm_yr_wk'], how = 'left')

In [23]:
train_clean_3['sell_price'] = train_clean_3['sell_price'].fillna(0)

In [28]:
print(train_clean_3['sell_price'].min())
print(train_clean_3['sell_price'].max())

0.0
107.32


In [32]:
train_clean_3.to_parquet('../data/interim/train_clean_3.parquet', index=False)

In [33]:
train_clean_3

,item_id,dept_id,cat_id,store_id,state_id,d,items_sold,date,wm_yr_wk,event_name,event_type,sell_price
0,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,NaN,NaN,0.00
1,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,NaN,NaN,0.00
2,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,NaN,NaN,0.00
3,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,NaN,NaN,0.00
4,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,NaN,NaN,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
47107045,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,d_1541,2,2015-04-18,11512,NaN,NaN,2.88
47107046,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,d_1541,0,2015-04-18,11512,NaN,NaN,2.00
47107047,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,d_1541,0,2015-04-18,11512,NaN,NaN,3.98
47107048,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,d_1541,0,2015-04-18,11512,NaN,NaN,1.28
